In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import tqdm
import pickle
import math

pd.set_option('display.max_columns', None)

In [ ]:
mobile = pd.read_csv('Z:\\event_mobile_ppl.tsv', sep='\t',
                    dtype={'account_id': str, 'content_section.1': str, 'download_link': str, 'role': str,
                         'get-legal-help': str, 'keywords': str, 'Outgoing_link': str, 'Language': str,
                         'legal_position': str}, 
                     parse_dates=['event_date'],
                     index_col='event_id')

In [ ]:
event_pairs = mobile.groupby('person_id').agg({'page_url': lambda x: set(tuple((a, b)) for a in x for b in x if a < b)})

In [ ]:
event_pairs

In [ ]:
all_pairs = [x for a in event_pairs.page_url for x in list(a)]

In [ ]:
from collections import Counter

In [ ]:
pair_counts = Counter(all_pairs)

In [ ]:
pair_counts.most_common()

In [ ]:
mobile['page_date'] = list(zip(*[mobile[c] for c in ['page_url', 'event_date']]))

In [ ]:
event_date = mobile.groupby('person_id').agg({'page_date': 
                                              lambda x: [tuple((a[0], b[0], 
                                                                (a[1]-b[1])/pd.Timedelta('00:00:01'))) 
                                                         for a in x for b in x if a[0] < b[0]]})

In [ ]:
event_date = mobile.groupby('person_id').agg({'page_date': 
                                              lambda x: set(tuple((a[0], b[0])) for a in x for b in x 
                                                            if b[1] - a[1] > pd.Timedelta('24:00:00'))})

In [ ]:
event_date

In [ ]:
pair_causal_counts = Counter([x for a in event_date.page_date for x in list(a)])

In [ ]:
pair_causal_counts.most_common()

In [ ]:
mobile_not_homepage = mobile[mobile['page_url'] != 'https://www.illinoislegalaid.org/']

In [ ]:
mobile_not_homepage

In [ ]:
event_date_diff = mobile_not_homepage.groupby('person_id').agg({'page_date': 
                                              lambda x: set(tuple((a[0], b[0])) for a in x for b in x 
                                                            if ((b[1] - a[1] > pd.Timedelta('24:00:00')) and
                                                               (a[0] != b[0])))})

In [ ]:
pair_different_causal_counts = Counter([x for a in event_date_diff.page_date for x in list(a)])

In [ ]:
pair_different_causal_counts.most_common()

In [ ]:
%%capture cap --no-stderr
print(pair_different_causal_counts.most_common())

In [ ]:
with open('pair_different_causal_counts.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
with open('pair_different_causal_counts.pkl', 'wb') as f:
    pickle.dump(pair_different_causal_counts, f)

In [ ]:
%%capture cap --no-stderr
print(pair_causal_counts.most_common())

In [ ]:
with open('pair_causal_counts.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
with open('pair_causal_counts.pkl', 'wb') as f:
    pickle.dump(pair_causal_counts, f)

In [ ]:
%%capture cap --no-stderr
print(pair_counts.most_common())

In [ ]:
with open('pair_counts.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
with open('pair_counts.pkl', 'wb') as f:
    pickle.dump(pair_counts, f)